In [30]:
!pip install nltk


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd

df=pd.read_csv("final.csv")
df.head(2)

start=0
end=10000

texts=df["Sentences"].values
labels=df["Labels"].values


In [2]:
# Import necessary libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC



In [3]:

# Convert labels to numerical values (0, 1, 2, 3)
label_mapping = {label: idx for idx, label in enumerate(set(labels))}
numeric_labels = [label_mapping[label] for label in labels]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, numeric_labels, test_size=0.2, random_state=42,stratify=numeric_labels)

# Create a CountVectorizer to convert text data into a bag-of-words representation
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)


In [4]:
label_mapping.keys()

dict_keys(['benefits', 'recruiting_process', 'equal_opportunity', 'about_company', 'job_functions', 'required_qualifications'])

In [5]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
logistic_regression = LogisticRegression(max_iter=1000, random_state=42)
naive_bayes = MultinomialNB()
svm_classifier = SVC(kernel='linear', probability=True, random_state=42)  # You can adjust the kernel and other parameters

# Create a StackingClassifier with Random Forest, Logistic Regression, Naive Bayes, and SVM
stacking_classifier = StackingClassifier(
    estimators=[
        ('random_forest', rf_classifier),
        ('logistic_regression', logistic_regression),
        ('naive_bayes', naive_bayes),
        ('svm', svm_classifier)
    ],
    final_estimator=LogisticRegression(),
    stack_method='auto',
    n_jobs=-1
)

# Train the stacking classifier
stacking_classifier.fit(X_train_vectorized, y_train)


In [37]:
type(X_train)

numpy.ndarray

In [38]:

model=stacking_classifier
# Make predictions on the test set
predictions = model.predict(X_test_vectorized)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")




Accuracy: 0.90


In [39]:
type(predictions)

numpy.ndarray

In [40]:
# Display classification report
print("Classification Report:")
print(classification_report(y_test, predictions, target_names=label_mapping.keys()))

Classification Report:
                         precision    recall  f1-score   support

          about_company       0.91      1.00      0.95        10
          job_functions       0.85      0.90      0.88        39
     recruiting_process       0.86      0.92      0.89        13
required_qualifications       0.93      0.87      0.90        46
               benefits       0.93      0.88      0.90        16

               accuracy                           0.90       124
              macro avg       0.90      0.91      0.90       124
           weighted avg       0.90      0.90      0.90       124



In [41]:
import pickle
model_filename = "text_classification_model.pkl"
with open(model_filename, 'wb') as model_file:
    pickle.dump(stacking_classifier, model_file)

print(f"Model saved to {model_filename}")

# Load the model back using pickle
with open(model_filename, 'rb') as model_file:
    model = pickle.load(model_file)

Model saved to text_classification_model.pkl


In [42]:
jd="Clariant is a focused and innovative specialty chemical company based near Basel in Switzerland. Last year the company recorded CHF 6.6 billion in sales harnessing the talents of its 18 000 employees across 53 countries. Clariant’s portfolio is optimized for future growth, with a focus in four business areas: Care Chemicals, Catalysts, Natural Resources, and Plastics & Coatings. We believe that our growth is all about our customers and their success. Investment in innovation capabilities and developing inspiring environments for courageous and determined minds are therefore key to reaching our goal. Join us and help create sustainable value through chemistry for customers all over the globe. The Research Engineer is responsible, as a member of the project team, for the evaluation of catalytic materials to develop new or improved commercial catalyst products. Responsibilities Operate, maintain, and troubleshoot catalyst test units and analytical tools used to discover new catalysts and processes that deliver value to SC businesses . Specifically, evaluate catalysts in high-throughput and conventional fixed bed and batch reactors using analytical hardware such as gas and liquid chromatography. Plan experiments in collaboration with the technical project team. Conduct equipment operational programming. Develop and implement GC methods and maintain equipment. Characterize catalysts using BET, XRD, LS, SEM, etc. and interpret resulting data. May prepare or assist in preparing catalytic materials. Work-up, analyze, and present data using appropriate specialty software packages such as DOE, visualization, and statistical analysis programs. Communicate results internally via reports, presentations, and informally. Generate IP and assist with patent preparation. Operate a safe laboratory by working closely with the site safety committee and coworkers Supervisory Responsibilities: • May direct technical/research assistants when applicable. Requirements BS in Chemical Engineering, Materials Science, or a related field with 3-8 years of industrial experience in heterogeneous catalysis, or (PHD or MS) in Chemical Engineering, Materials Science, or a related field with 1-5 years of industrial experience in heterogeneous catalysis Skilled in the evaluation of solid catalysts in fixed bed and batch reactor systems. Knowledgeable in the use of automated laboratory hardware and software , and in data analysis and reduction software and techniques. Experience with analytical methods such as XRD, SEM, TEM, XPS, BET, IR, TPD\TPR, HPLC, GC , MS, etc.; including method development. Record of success (commercialized products, patents, publications) conducting team-oriented catalyst research. Senior technical team member with responsibilities in one or more projects; ability to execute aspects of multiple projects simultaneously. Serve as a mentor and technical resource to other technical staff. Superior creativity and imagination with respect to new products, processes, procedures Strong communication, interpersonal, and teamwork skills with extended project teams on HT projects. Expertise in high throughput R&D methods. Optionally experienced in catalyst synthesis Learn more about Clariant Follow us on Facebook , Twitter and LinkedIn Our commitment to people - download our Employment Standards Brochure"


In [43]:
import re
def tokenize_paragraph(paragraph):
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', paragraph)
        return sentences



In [44]:
import json
import nltk
from nltk.tokenize import sent_tokenize


def predict(paragraph, model, vectorizer, label_mapping, confidence_threshold=0.3, extra_label='extra'):
   
    sentences = sent_tokenize(paragraph)
    sentences_vectorized = vectorizer.transform(sentences)    
    sentence_predictions_proba = model.predict_proba(sentences_vectorized)


    labels=list(label_mapping.keys())
    labels.append('extra')

    result_dict = {label: [] for label in labels}
    
   
    for sentence, label, proba in zip(sentences, model.predict(sentences_vectorized), sentence_predictions_proba.max(axis=1)):
        if proba >= confidence_threshold:
            result_dict[list(label_mapping.keys())[label]].append(sentence)
        else:
            result_dict[extra_label].append(sentence)

    result_json = json.dumps(result_dict, indent=2)

    return result_json


# Download NLTK sentence tokenizer data (run once)
# nltk.download('punkt')

result = predict(jd, model, vectorizer, label_mapping)
print(result)


{
  "about_company": [
    "Clariant is a focused and innovative specialty chemical company based near Basel in Switzerland.",
    "Last year the company recorded CHF 6.6 billion in sales harnessing the talents of its 18 000 employees across 53 countries.",
    "Clariant\u2019s portfolio is optimized for future growth, with a focus in four business areas: Care Chemicals, Catalysts, Natural Resources, and Plastics & Coatings.",
    "We believe that our growth is all about our customers and their success.",
    "Investment in innovation capabilities and developing inspiring environments for courageous and determined minds are therefore key to reaching our goal.",
    "The Research Engineer is responsible, as a member of the project team, for the evaluation of catalytic materials to develop new or improved commercial catalyst products.",
    "Responsibilities Operate, maintain, and troubleshoot catalyst test units and analytical tools used to discover new catalysts and processes that deli